In [1]:
import random 
from sage.all import Graph, RR, infinity        #zakaj to podčrta
import warnings
warnings.filterwarnings("ignore", message=".*Signature.*numpy.longdouble.*")

In [2]:
# graphs.RandomTree(7).show()

# OSNOVNE FUNKCIJE

In [3]:
# definiramo funkcijo za Somborjev indeks

def general_sombor_index(G, alpha):
    alpha = RR(alpha)
    deg = G.degree()  
    val = 0         #RR(0)
    for u, v in G.edges(labels=False):
        du = deg[u]
        dv = deg[v]
        val += (du**2 + dv**2)**alpha
    return RR(val)

# definirajmo funkcijo za največjo stopnjo - delto

def max_degree(G):
    return max(G.degree(v) for v in G.vertices()) if G.num_verts() > 0 else 0

# MAJHNI GRAFI (n≤10)

In [4]:
# funkcija, ki vrne vse variacije drevesa z n vozlišči in vsemi možnimi deltami

# Funkcija generira vsa neizomorfna drevesa z n vozlišči, vsako drevo razvrsti glede na njegovo maksimalno stopnjo (Δ) ter vrne slovar, 
# kjer je ključ Δ, vrednost pa seznam dreves z isto maksimalno stopnjo.


def classify_trees_by_max_degree(n):
    """
    Vrne slovar:
        Δ -> [seznam neizomorfnih dreves kot Sage Graph objekti]
    """
    result = {}

    for T in graphs.trees(n):
        # izračun maximalne stopnje – kompatibilno z vsemi verzijami Sage
        delta = max(T.degree(v) for v in T.vertices()) if T.num_verts() > 0 else 0

        if delta not in result:
            result[delta] = []

        result[delta].append(T)

    return result

In [5]:
# funkcija, ki vrne drevo z največjim indeksom iz seznama vseh variacij

def systematic_best_trees_all(n, alpha, return_all_best=True):
    best_val = -infinity
    best_list = []   # tukaj bomo shranili: (edges, delta)

    trees_by_delta = classify_trees_by_max_degree(n)

    for delta in trees_by_delta:
        for T in trees_by_delta[delta]:

            val = general_sombor_index(T, alpha)
            edges = T.edges(labels=False)
            maxdeg = max(T.degree(v) for v in T.vertices())

            # Nova najboljša vrednost
            if val > best_val + 1e-12:
                best_val = val
                best_list = [(edges, maxdeg)]

            # Če je enako najboljši in želimo vse najboljše
            elif abs(val - best_val) <= 1e-12 and return_all_best:
                best_list.append((edges, maxdeg))

    return best_val, best_list

In [6]:
# funkcija, ki za vsak delta zračuna pri katerem drevesu je najboljši somborjev indeks

def best_sombor_by_delta(n, alpha):
    """
    Za vsak Δ izračuna največji Somborjev indeks in vrne
    najboljša drevesa ter njihove Δ in robove.

    Vrne slovar:
    {
        Δ : {
                "best_val": vrednost Sombor indeksa,
                "best_trees": [ (edges, delta), ... ]
            }
    }
    """
    result = {}
    trees_by_delta = classify_trees_by_max_degree(n)

    for delta, tree_list in trees_by_delta.items():

        best_val = -infinity
        best_trees = []

        for T in tree_list:

            val = general_sombor_index(T, alpha)
            edges = T.edges(labels=False)
            maxdeg = max(T.degree(v) for v in T.vertices())

            # ali je to novo najboljše drevo pri tem Δ?
            if val > best_val + 1e-12:
                best_val = val
                best_trees = [(edges, maxdeg)]

            elif abs(val - best_val) <= 1e-12:
                best_trees.append((edges, maxdeg))

        result[delta] = {
            "best_val": best_val,
            "best_trees": best_trees
        }

    return result

## Uporabljene alfe

In [7]:
alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]

In [8]:
#napišemo funkcijo, da ni treba vsakič vsega znova pisat
def systematic_testing(n, alpha_values):
    """
    Run systematic + delta-based Sombor search for a fixed n
    and multiple alpha values.

    Assumes the following functions already exist:
      - best_sombor_by_delta(n, alpha)
      - systematic_best_trees_all(n, alpha)
    """

    for alpha in alpha_values:
        alpha_R = RR(alpha)

        print("=" * 60)
        print(f"α = {float(alpha_R):.2f}")
        print("=" * 60)

        # 1. Compute best Sombor index for each Δ
        res = best_sombor_by_delta(n, alpha_R)

        # 2. Print results by Δ
        for delta in sorted(res.keys()):
            print(f"Δ = {delta}")
            print("  največji Sombor indeks =", res[delta]["best_val"])
            for edges, d in res[delta]["best_trees"]:
                print("  edges:", edges)
            print()

        # 3. Find global maximum over all trees
        best_global_val, best_global_list = systematic_best_trees_all(n, alpha_R)

        # 4. Determine Δ where the global maximum occurs
        max_delta = best_global_list[0][1]  # (edges, delta)

        print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha_R):.2f}")
        print(f"   Δ = {max_delta},  Sombor indeks = {best_global_val}")

        # 5. Print globally optimal trees
        for edges, d in best_global_list:
            print("   najboljše drevo, edges:", edges)

        print("\n")


## n=2

In [9]:
systematic_testing(2, alpha_values)

α = 0.05
Δ = 1
  največji Sombor indeks = 1.03526492384138
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 1,  Sombor indeks = 1.03526492384138
   najboljše drevo, edges: [(0, 1)]


α = 0.45
Δ = 1
  največji Sombor indeks = 1.36604025675440
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 1,  Sombor indeks = 1.36604025675440
   najboljše drevo, edges: [(0, 1)]


α = 0.50
Δ = 1
  največji Sombor indeks = 1.41421356237310
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 1,  Sombor indeks = 1.41421356237310
   najboljše drevo, edges: [(0, 1)]


α = 0.55
Δ = 1
  največji Sombor indeks = 1.46408569594563
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.55
   Δ = 1,  Sombor indeks = 1.46408569594563
   najboljše drevo, edges: [(0, 1)]


α = 0.95
Δ = 1
  največji Sombor indeks = 1.93187265784969
  edges: [(0, 1)]

→ GLOBALNI MAKSIMUM pri α = 0.95
   Δ = 1,  Sombor indeks = 1.93187265784969
   najboljše drevo, edges: [(0, 1)]




In [10]:
#ne še zbrisat za vsak slučaj 
"""
n = 2

for alpha in alpha_values:
    alpha_R = RR(alpha)

    print("="*60)
    print(f"α = {float(alpha_R):.2f}")
    print("="*60)

    # 1. izračun za vsak Δ
    res = best_sombor_by_delta(n, alpha_R)

    # izpis rezultatov po Δ
    for delta in sorted(res.keys()):
        print(f"Δ = {delta}")
        print("  največji Sombor indeks =", res[delta]["best_val"])
        for edges, d in res[delta]["best_trees"]:
            print("  edges:", edges)
        print()

    # 2. poišči globalni maksimum med vsemi drevesi pri tem α
    best_global_val, best_global_list = systematic_best_trees_all(n, alpha_R)

    # 3. določimo pri katerem Δ ta maksimum nastopi
    max_delta = best_global_list[0][1]   # drugi element v (edges, delta)

    print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha_R):.2f}")
    print(f"   Δ = {max_delta},  Sombor indeks = {best_global_val}")

    # izpišemo najboljše drevo/drevesa
    for edges, d in best_global_list:
        print("   najboljše drevo, edges:", edges)

    print("\n")

"""

'\nn = 2\n\nfor alpha in alpha_values:\n    alpha_R = RR(alpha)\n\n    print("="*60)\n    print(f"α = {float(alpha_R):.2f}")\n    print("="*60)\n\n    # 1. izračun za vsak Δ\n    res = best_sombor_by_delta(n, alpha_R)\n\n    # izpis rezultatov po Δ\n    for delta in sorted(res.keys()):\n        print(f"Δ = {delta}")\n        print("  največji Sombor indeks =", res[delta]["best_val"])\n        for edges, d in res[delta]["best_trees"]:\n            print("  edges:", edges)\n        print()\n\n    # 2. poišči globalni maksimum med vsemi drevesi pri tem α\n    best_global_val, best_global_list = systematic_best_trees_all(n, alpha_R)\n\n    # 3. določimo pri katerem Δ ta maksimum nastopi\n    max_delta = best_global_list[0][1]   # drugi element v (edges, delta)\n\n    print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha_R):.2f}")\n    print(f"   Δ = {max_delta},  Sombor indeks = {best_global_val}")\n\n    # izpišemo najboljše drevo/drevesa\n    for edges, d in best_global_list:\n        print("

## n=3

In [11]:
systematic_testing(3, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 2.16759677346874
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 2,  Sombor indeks = 2.16759677346874
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.45
Δ = 2
  največji Sombor indeks = 4.12635413536159
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 2,  Sombor indeks = 4.12635413536159
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.50
Δ = 2
  največji Sombor indeks = 4.47213595499958
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 2,  Sombor indeks = 4.47213595499958
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.55
Δ = 2
  največji Sombor indeks = 4.84689373328531
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.55
   Δ = 2,  Sombor indeks = 4.84689373328531
   najboljše drevo, edges: [(0, 1), (0, 2)]


α = 0.95
Δ = 2
  največji Sombor indeks = 9.22680834590588
  edges: [(0, 1), (0, 2)]

→ GLOBALNI MAKSIMUM pri α = 0.95
   Δ = 2,  Sombor indeks = 9.22680834590588
 

## n=4

In [12]:
systematic_testing(4, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 3.27716624553658
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 3.36605536290589
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 3,  Sombor indeks = 3.36605536290589
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.45
Δ = 2
  največji Sombor indeks = 6.67547539000011
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 8.45514879379336
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 3,  Sombor indeks = 8.45514879379336
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.50
Δ = 2
  največji Sombor indeks = 7.30056307974577
  edges: [(0, 1), (0, 3), (1, 2)]

Δ = 3
  največji Sombor indeks = 9.48683298050514
  edges: [(0, 1), (0, 2), (0, 3)]

→ GLOBALNI MAKSIMUM pri α = 0.50
   Δ = 3,  Sombor indeks = 9.48683298050514
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3)]


α = 0.55
Δ = 2
  največji Sombor indeks = 7.98523012487231
  edges: [(

## n=5

In [13]:
systematic_testing(5, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 4.38673571760443
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 4.46466959319389
  edges: [(0, 1), (0, 3), (0, 4), (1, 2)]

Δ = 4
  največji Sombor indeks = 4.60874243131641
  edges: [(0, 1), (0, 2), (0, 3), (0, 4)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 4,  Sombor indeks = 4.60874243131641
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4)]


α = 0.45
Δ = 2
  največji Sombor indeks = 9.22459664463864
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 10.8715144444270
  edges: [(0, 1), (0, 3), (0, 4), (1, 2)]

Δ = 4
  največji Sombor indeks = 14.3140327308418
  edges: [(0, 1), (0, 2), (0, 3), (0, 4)]

→ GLOBALNI MAKSIMUM pri α = 0.45
   Δ = 4,  Sombor indeks = 14.3140327308418
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4)]


α = 0.50
Δ = 2
  največji Sombor indeks = 10.1289902044920
  edges: [(0, 1), (0, 3), (1, 2), (3, 4)]

Δ = 3
  največji Sombor indeks = 12.16617457

## n=6

In [14]:
systematic_testing(6, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 5.49630518967227
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5)]

Δ = 3
  največji Sombor indeks = 5.64355698997166
  edges: [(0, 1), (0, 4), (0, 5), (1, 2), (1, 3)]

Δ = 4
  največji Sombor indeks = 5.70194155986322
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (1, 2)]

Δ = 5
  največji Sombor indeks = 5.88462336394871
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 5,  Sombor indeks = 5.88462336394871
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]


α = 0.45
Δ = 2
  največji Sombor indeks = 11.7737178992772
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5)]

Δ = 3
  največji Sombor indeks = 14.9452776986243
  edges: [(0, 1), (0, 4), (0, 5), (1, 2), (1, 3)]

Δ = 4
  največji Sombor indeks = 16.6487261588689
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (1, 2)]

Δ = 5
  največji Sombor indeks = 21.6624718279813
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

→ GLOBALNI MAKSIMUM pri α =

## n=7

In [15]:
systematic_testing(7, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 6.60587466174012
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5), (5, 6)]

Δ = 3
  največji Sombor indeks = 6.76174241291905
  edges: [(0, 1), (0, 4), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 6.87521267517925
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (1, 2), (1, 3)]

Δ = 5
  največji Sombor indeks = 6.97486529247889
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (1, 2)]

Δ = 6
  največji Sombor indeks = 7.18722658362132
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 6,  Sombor indeks = 7.18722658362132
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]


α = 0.45
Δ = 2
  največji Sombor indeks = 14.3228391539157
  edges: [(0, 1), (0, 4), (1, 2), (2, 3), (4, 5), (5, 6)]

Δ = 3
  največji Sombor indeks = 17.6166747534923
  edges: [(0, 1), (0, 4), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 20.6289900232642
  edges: [(0, 

## n=8

In [16]:
systematic_testing(8, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 7.71544413380796
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7)]

Δ = 3
  največji Sombor indeks = 7.92105861703743
  edges: [(0, 1), (0, 4), (0, 7), (1, 2), (1, 3), (4, 5), (4, 6)]

Δ = 4
  največji Sombor indeks = 8.10232076197734
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4)]

Δ = 5
  največji Sombor indeks = 8.14455294530322
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3)]

Δ = 6
  največji Sombor indeks = 8.27570347688230
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2)]

Δ = 7
  največji Sombor indeks = 8.51229253461060
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 7,  Sombor indeks = 8.51229253461060
   najboljše drevo, edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]


α = 0.45
Δ = 2
  največji Sombor indeks = 16.8719604085542
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7)]



## n=9

In [17]:
systematic_testing(9, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 8.82501360587581
  edges: [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7), (7, 8)]

Δ = 3
  največji Sombor indeks = 9.03924403998482
  edges: [(0, 1), (0, 5), (0, 8), (1, 2), (2, 3), (2, 4), (5, 6), (5, 7)]

Δ = 4
  največji Sombor indeks = 9.23628634625770
  edges: [(0, 1), (0, 5), (1, 2), (1, 3), (1, 4), (5, 6), (5, 7), (5, 8)]

Δ = 5
  največji Sombor indeks = 9.36829073992565
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2), (1, 3), (1, 4)]

Δ = 6
  največji Sombor indeks = 9.44304489036981
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2), (1, 3)]

Δ = 7
  največji Sombor indeks = 9.59963895871060
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 2)]

Δ = 8
  največji Sombor indeks = 9.85679342400157
  edges: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]

→ GLOBALNI MAKSIMUM pri α = 0.05
   Δ = 8,  Sombor indeks = 9.85679342400157
   najboljše drevo, edges: [(0, 1), (0,

## n=10

In [18]:
systematic_testing(10, alpha_values)

α = 0.05
Δ = 2
  največji Sombor indeks = 9.93458307794365
  edges: [(0, 1), (0, 6), (1, 2), (2, 3), (3, 4), (4, 5), (6, 7), (7, 8), (8, 9)]

Δ = 3
  največji Sombor indeks = 10.1985602441032
  edges: [(0, 1), (0, 6), (0, 9), (1, 2), (1, 5), (2, 3), (2, 4), (6, 7), (6, 8)]
  edges: [(0, 1), (0, 4), (0, 7), (1, 2), (1, 3), (4, 5), (4, 6), (7, 8), (7, 9)]

Δ = 4
  največji Sombor indeks = 10.3843699874526
  edges: [(0, 1), (0, 5), (0, 9), (1, 2), (1, 3), (1, 4), (5, 6), (5, 7), (5, 8)]

Δ = 5
  največji Sombor indeks = 10.6314391729766
  edges: [(0, 1), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5)]

Δ = 6
  največji Sombor indeks = 10.6643411415811
  edges: [(0, 1), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4)]

Δ = 7
  največji Sombor indeks = 10.7653872571051
  edges: [(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3)]

Δ = 8
  največji Sombor indeks = 10.9433745911232
  edges: [(0, 1), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0,

# VELIKI GRAFI (n>10)

In [19]:
"""
Docstring for naloga.ipynb
alfa=0.1
res = best_sombor_by_delta(2, alpha=0.1)

for delta in sorted(res.keys()):
    print(f"Δ = {delta}")
    print("  največji Sombor indeks =", res[delta]["best_val"])
    for edges, d in res[delta]["best_trees"]:
        print("  edges:", edges)
    print()
"""

'\nDocstring for naloga.ipynb\nalfa=0.1\nres = best_sombor_by_delta(2, alpha=0.1)\n\nfor delta in sorted(res.keys()):\n    print(f"Δ = {delta}")\n    print("  največji Sombor indeks =", res[delta]["best_val"])\n    for edges, d in res[delta]["best_trees"]:\n        print("  edges:", edges)\n    print()\n'

In [20]:
"""
# alfa=0.1
res = best_sombor_by_delta(3, alpha=0.1)

for delta in sorted(res.keys()):
    print(f"Δ = {delta}")
    print("  največji Sombor indeks =", res[delta]["best_val"])
    for edges, d in res[delta]["best_trees"]:
        print("  edges:", edges)
    print()"""

'\n# alfa=0.1\nres = best_sombor_by_delta(3, alpha=0.1)\n\nfor delta in sorted(res.keys()):\n    print(f"Δ = {delta}")\n    print("  največji Sombor indeks =", res[delta]["best_val"])\n    for edges, d in res[delta]["best_trees"]:\n        print("  edges:", edges)\n    print()'

In [21]:
#testiranje na primeru
G = graphs.PathGraph(4)
#G.show()

print("vozlišča:", G.vertices())
print("povezave:", G.edges(labels=False))
print("stopnje:", G.degree())

alpha_values = [0.1, 0.5, 0.9]

for alpha in alpha_values:
    val = general_sombor_index(G, alpha)
    print(f"  α = {alpha:.2f}  →  SO_α(G) = {val}")


vozlišča: [0, 1, 2, 3]
povezave: [(0, 1), (1, 2), (2, 3)]
stopnje: [1, 2, 2, 1]
  α = 0.10  →  SO_α(G) = 3.58038229952095
  α = 0.50  →  SO_α(G) = 7.30056307974577
  α = 0.90  →  SO_α(G) = 15.0114183960577


In [22]:
# funkcija, ki išče najboljša drevesa glede na Somborjev indeks
""""
def systematic_best_trees(n, Delta, alpha, return_all_best=True):
    best_val = -infinity
    best_trees = []
    
    for T in graphs.trees(n):
        if max_degree(T) > Delta:
            continue
        
        val = general_sombor_index(T, alpha)
        if val > best_val + 1e-12:
            best_val = val
            best_trees = [T]
        elif abs(val - best_val) <= 1e-12 and return_all_best:
            best_trees.append(T)
    
    return best_val, best_trees
"""

'"\ndef systematic_best_trees(n, Delta, alpha, return_all_best=True):\n    best_val = -infinity\n    best_trees = []\n    \n    for T in graphs.trees(n):\n        if max_degree(T) > Delta:\n            continue\n        \n        val = general_sombor_index(T, alpha)\n        if val > best_val + 1e-12:\n            best_val = val\n            best_trees = [T]\n        elif abs(val - best_val) <= 1e-12 and return_all_best:\n            best_trees.append(T)\n    \n    return best_val, best_trees\n'

In [23]:
#testiranje na primeru
#zakomentiramo ker je funkcija zakomentirana in potem to ne dela
"""n = 10    
Delta = 3    
alpha = 1  

best_val, best_trees = systematic_best_trees(n, Delta, alpha)

print(f"SO_{alpha:.2f}(G) = {best_val}\n")

print("Ekstremi: \n")
for i, T in enumerate(best_trees, 1):
    print(f"Drevo {i}:")
    print("stopnje vozlišč:", sorted(T.degree()))
    print("max stopnja:", max_degree(T))
    print("povezave:", T.edges(labels=False))
    T.show()  
"""

'n = 10    \nDelta = 3    \nalpha = 1  \n\nbest_val, best_trees = systematic_best_trees(n, Delta, alpha)\n\nprint(f"SO_{alpha:.2f}(G) = {best_val}\n")\n\nprint("Ekstremi: \n")\nfor i, T in enumerate(best_trees, 1):\n    print(f"Drevo {i}:")\n    print("stopnje vozlišč:", sorted(T.degree()))\n    print("max stopnja:", max_degree(T))\n    print("povezave:", T.edges(labels=False))\n    T.show()  \n'

In [24]:
#alpha = 0.5
#P5 = graphs.PathGraph(5)
#Star = Graph()
#Star.add_vertices(range(5))
#Star.add_edges([(0,1), (0,2), (0,3), (0,4)])
#print("Degrees P5:   ", P5.degree())
#print("Degrees Star: ", Star.degree())
#print("\nGeneral Sombor index values (alpha = 0.5):")
#print("  SO_0.5(P5)   =", general_sombor_index(P5, alpha))
#print("  SO_0.5(Star) =", general_sombor_index(Star, alpha))
#P5.show()
#Star.show()


In [25]:
# Random tree with bounded maximum degree Δ
# funkcija, ki vrne drevo n vozlišč in največjo stopnjo manjše ali enako delta

# Z DELTA
"""
def random_bounded_degree_tree(n, Delta, max_tries=20):
    
    Generate a random n-vertex tree with maximum degree <= Delta.
    Simple incremental construction: start with one vertex and
    keep attaching new vertices to random existing vertices whose
    degree is still < Delta.
    
    Assumes Delta >= 2 and n >= 1.
    
    #if n == 1:
    #    G = Graph()
    #    G.add_vertex(0)
    #   return G
    
    for _ in range(max_tries):
        G = Graph()
        G.add_vertex(0)

        ok = True
        for v in range(1, n):
            kandidati = [u for u in G.vertices() if G.degree(u) < Delta]
            if not kandidati:
                ok = False
                break
            parent = random.choice(kandidati)
            G.add_vertex(v)
            G.add_edge(v, parent)

        if ok and G.is_tree():
            return G
"""

'\ndef random_bounded_degree_tree(n, Delta, max_tries=20):\n    \n    Generate a random n-vertex tree with maximum degree <= Delta.\n    Simple incremental construction: start with one vertex and\n    keep attaching new vertices to random existing vertices whose\n    degree is still < Delta.\n    \n    Assumes Delta >= 2 and n >= 1.\n    \n    #if n == 1:\n    #    G = Graph()\n    #    G.add_vertex(0)\n    #   return G\n    \n    for _ in range(max_tries):\n        G = Graph()\n        G.add_vertex(0)\n\n        ok = True\n        for v in range(1, n):\n            kandidati = [u for u in G.vertices() if G.degree(u) < Delta]\n            if not kandidati:\n                ok = False\n                break\n            parent = random.choice(kandidati)\n            G.add_vertex(v)\n            G.add_edge(v, parent)\n\n        if ok and G.is_tree():\n            return G\n'

In [26]:
#Local move on trees (for stochastic search)
# local move by relocating a leaf

# Z DELTA

def random_tree_move(G, Delta, max_attempts=50):
    
    """Simpler local move:
      - pick a random leaf ℓ with neighbor p,
      - detach ℓ from p,
      - attach ℓ to a different vertex q with degree < Delta.
    This keeps the graph a tree and respects the max degree constraint.
    """
    H = G.copy()

    for _ in range(max_attempts):
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        if not leaves:
            return G

        leaf = random.choice(leaves)
        parent = H.neighbors(leaf)[0]

        candidates = [
            v for v in H.vertices()
            if v != leaf and v != parent and H.degree(v) < Delta
        ]

        if not candidates:
            continue

        new_parent = random.choice(candidates)

        H.delete_edge(leaf, parent)
        H.add_edge(leaf, new_parent)

        if H.is_tree():
            return H

        # rollback
        H.delete_edge(leaf, new_parent)
        H.add_edge(leaf, parent)

    return G



In [27]:

#Stochastic hill-climbing search (for larger n)

"""
def stochastic_best_tree(n, Delta, alpha, n_starts=None, n_steps=None, start_tree=None):
    
    Stochastic search for trees maximizing the general Sombor index
    among n-vertex trees with maximum degree <= Delta.
    
    Uses multiple random restarts + simple hill-climbing (accepting moves
    that do not decrease SO_alpha).
    
    Parameters
    ----------
    n : int
    Delta : int
    alpha : float
    n_starts : int
        Number of random initial trees (multi-start).
    n_steps : int
        Number of local moves per start.
    verbose : bool
        If True, prints progress information.
        
    Returns
    -------
    (best_val, best_tree)
        best_val : RR
        best_tree : Graph
    
    if n_starts is None:
        n_starts = min(n // 2, 30)

    if n_steps is None:
        n_steps = min(35 * n, 6000)

    best_val = -infinity
    best_tree = None

    alpha = RR(alpha)

    for s in range(n_starts):

        if s == 0 and start_tree is not None:
            G = start_tree.copy()
        else:
            G = random_bounded_degree_tree(n, Delta)

        val = general_sombor_index(G, alpha)
        local_best_val = val
        local_best_tree = G

        for _ in range(n_steps):
            H = random_tree_move(G, Delta)
            if H is G:
                continue

            new_val = general_sombor_index(H, alpha)

            if new_val >= val - 1e-12:
                G, val = H, new_val
                if val > local_best_val + 1e-12:
                    local_best_val = val
                    local_best_tree = G

        if local_best_val > best_val + 1e-12:
            best_val = local_best_val
            best_tree = local_best_tree

    return best_val, best_tree
"""

'\ndef stochastic_best_tree(n, Delta, alpha, n_starts=None, n_steps=None, start_tree=None):\n    \n    Stochastic search for trees maximizing the general Sombor index\n    among n-vertex trees with maximum degree <= Delta.\n    \n    Uses multiple random restarts + simple hill-climbing (accepting moves\n    that do not decrease SO_alpha).\n    \n    Parameters\n    ----------\n    n : int\n    Delta : int\n    alpha : float\n    n_starts : int\n        Number of random initial trees (multi-start).\n    n_steps : int\n        Number of local moves per start.\n    verbose : bool\n        If True, prints progress information.\n        \n    Returns\n    -------\n    (best_val, best_tree)\n        best_val : RR\n        best_tree : Graph\n    \n    if n_starts is None:\n        n_starts = min(n // 2, 30)\n\n    if n_steps is None:\n        n_steps = min(35 * n, 6000)\n\n    best_val = -infinity\n    best_tree = None\n\n    alpha = RR(alpha)\n\n    for s in range(n_starts):\n\n        if s 

In [28]:
"""
alpha = 0.5
n = 55

val0, T0 = stochastic_best_tree(n, Delta=n-1, alpha=alpha,
                                n_starts=3, n_steps=1500)

Delta_star = max(T0.degree())
print("Naravni Δ* ≈", Delta_star)
"""

'\nalpha = 0.5\nn = 55\n\nval0, T0 = stochastic_best_tree(n, Delta=n-1, alpha=alpha,\n                                n_starts=3, n_steps=1500)\n\nDelta_star = max(T0.degree())\nprint("Naravni Δ* ≈", Delta_star)\n'

In [29]:
"""
Delta_candidates = sorted(set([
    Delta_star-5, Delta_star-2, Delta_star,
    Delta_star+2, Delta_star+5
]))

Delta_candidates = [d for d in Delta_candidates if d >= 3]
"""

'\nDelta_candidates = sorted(set([\n    Delta_star-5, Delta_star-2, Delta_star,\n    Delta_star+2, Delta_star+5\n]))\n\nDelta_candidates = [d for d in Delta_candidates if d >= 3]\n'

In [30]:
"""

n = 55
alpha = 0.5
prev_tree = T0

for Delta in Delta_candidates:
    val, T = stochastic_best_tree(
        n, Delta, alpha,
        n_starts=3,
        n_steps=800,
        start_tree=prev_tree
    )

    print(f"Δ = {Delta}, SO ≈ {val}")
    prev_tree = T
"""

'\n\nn = 55\nalpha = 0.5\nprev_tree = T0\n\nfor Delta in Delta_candidates:\n    val, T = stochastic_best_tree(\n        n, Delta, alpha,\n        n_starts=3,\n        n_steps=800,\n        start_tree=prev_tree\n    )\n\n    print(f"Δ = {Delta}, SO ≈ {val}")\n    prev_tree = T\n'

In [31]:
#stochastic_best_tree(55, 35, alpha=0.5)

In [32]:
#stochastic_best_tree(55, 25, alpha=0.5)

In [33]:
#stochastic_best_tree(55, 54, alpha=0.5)

In [34]:
"""
#ful dolg rab
n = 55
alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]
#alpha_values = [0.5]
Delta_values = [10, 15, 20, 25, 30, 35, 40, 45, 50, 54]

for alpha in alpha_values:
    alpha_R = RR(alpha)

    print("="*60)
    print(f"α = {float(alpha_R):.2f}")
    print("="*60)

    best_global_val = -infinity
    best_global_delta = None

    for Delta in Delta_values:
        val, T = stochastic_best_tree(
            n, Delta, alpha,)

        achieved_delta = max(T.degree())

        print(f"Δ = {Delta}")
        print("  največji Sombor indeks ≈", val)
        print("  dosežen max Δ =", achieved_delta)
        print()

        if val > best_global_val:
            best_global_val = val
            best_global_delta = achieved_delta


    print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha):.2f}")
    print(f"   Δ = {best_global_delta},  Sombor indeks ≈ {best_global_val}")
    print("\n")
"""

'\n#ful dolg rab\nn = 55\nalpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]\n#alpha_values = [0.5]\nDelta_values = [10, 15, 20, 25, 30, 35, 40, 45, 50, 54]\n\nfor alpha in alpha_values:\n    alpha_R = RR(alpha)\n\n    print("="*60)\n    print(f"α = {float(alpha_R):.2f}")\n    print("="*60)\n\n    best_global_val = -infinity\n    best_global_delta = None\n\n    for Delta in Delta_values:\n        val, T = stochastic_best_tree(\n            n, Delta, alpha,)\n\n        achieved_delta = max(T.degree())\n\n        print(f"Δ = {Delta}")\n        print("  največji Sombor indeks ≈", val)\n        print("  dosežen max Δ =", achieved_delta)\n        print()\n\n        if val > best_global_val:\n            best_global_val = val\n            best_global_delta = achieved_delta\n\n\n    print(f"→ GLOBALNI MAKSIMUM pri α = {float(alpha):.2f}")\n    print(f"   Δ = {best_global_delta},  Sombor indeks ≈ {best_global_val}")\n    print("\n")\n'

In [35]:
# funkcija ki vrne drevo n vozlišč in največjo stopnjo delta

# Z DELTA

def random_tree_with_exact_max_degree(n, Delta):
    """
    Generate a random tree on n vertices whose maximum degree is EXACTLY Delta.
    SageMath version (Graph()).

    Requires n >= Delta + 1 and Delta >= 2.
    """

    if Delta < 2:
        raise ValueError("Delta must be at least 2.")
    if n < Delta + 1:
        raise ValueError("To achieve max degree = Delta, n must be >= Delta + 1.")

    G = Graph()
    G.add_vertices(range(n))

    deg = {v: 0 for v in range(n)}

    # 1) Create a star to guarantee max degree = Delta
    center = 0
    for leaf in range(1, Delta + 1):
        G.add_edge(center, leaf)
        deg[center] += 1
        deg[leaf] += 1

    next_vertex = Delta + 1

    # 2) Add remaining vertices
    while next_vertex < n:
        # Candidates are only already-created vertices
        candidatos = [u for u in range(next_vertex) if deg[u] < Delta]

        if not candidatos:
            raise RuntimeError("No valid parent available (should not happen).")

        parent = random.choice(candidatos)
        child = next_vertex

        G.add_edge(parent, child)
        deg[parent] += 1
        deg[child] += 1

        next_vertex += 1

    # Check
    if max(deg.values()) != Delta:
        raise RuntimeError("Tree does not have max degree = Delta (BUG).")

    return G

In [36]:
def stochastic_search_fixed_n(n, Delta, alpha,
                              n_starts=None,
                              n_steps=None,
                              start_tree=None):
    """
    Stochastic hill-climbing search for trees maximizing SO_alpha
    among n-vertex trees with maximum degree EXACTLY Delta.

    Parameters
    ----------
    n : int
    Delta : int
    alpha : float in (0,1)
    n_starts : int or None
        Number of random restarts (default: min(n//2, 30))
    n_steps : int or None
        Local moves per restart (default: min(35*n, 6000))

    Returns
    -------
    (best_value, best_tree)
    """

    if n_starts is None:
        n_starts = min(n // 2, 30)

    if n_steps is None:
        n_steps = min(35 * n, 6000)

    alpha = RR(alpha)

    best_val = -infinity
    best_tree = None

    for _ in range(n_starts):

        # start from a tree with Δ(T)=Δ
        G = random_tree_with_exact_max_degree(n, Delta)
        val = general_sombor_index(G, alpha)

        for _ in range(n_steps):

            H = random_tree_move(G, Delta)
            if H is G:
                continue

            # enforce exact maximum degree
            if max_degree(H) != Delta:
                continue

            new_val = general_sombor_index(H, alpha)

            # hill-climbing (monotone)
            if new_val > val + 1e-12:
                G, val = H, new_val

        # update global best
        if val > best_val + 1e-12:
            best_val = val
            best_tree = G.copy()

    return best_val, best_tree


In [37]:
"""
n = 55
Delta = 25
alpha = 0.5

best_val, best_tree = stochastic_search_fixed_n(n, Delta, alpha)

print("Best Sombor index:", best_val)
print("Maximum degree:", max_degree(best_tree))

"""

'\nn = 55\nDelta = 25\nalpha = 0.5\n\nbest_val, best_tree = stochastic_search_fixed_n(n, Delta, alpha)\n\nprint("Best Sombor index:", best_val)\nprint("Maximum degree:", max_degree(best_tree))\n\n'

In [38]:
# dodamo to da naslednja funkcija dela hitreje 
def adaptive_params(n, Delta):
    if Delta <= 3:
        return 3, 600
    elif Delta <= 6:
        return 5, 1200
    elif Delta <= 10:
        return 7, 2000
    elif Delta <= n//2:
        return 9, 3000
    else:
        return 6, 2000

#vrne n_starts in n_steps glede na n in Delta

In [39]:
# za n>10
def scan_alpha_and_delta(
    n,
    alpha_values=None,
    Delta_values=None,
    n_starts=None,
    n_steps=None,
    print_edges=True):
    
    """
    Stochastic search for fixed n.
    If alpha_values or Delta_values are None, reasonable defaults are used.
    """

    # privzete vrednosti
    if alpha_values is None:
        alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]

    if Delta_values is None:
        Delta_values = [
            2,          # pot
            3, 4, 5,    # prvo razvejanje
            10,         # srednja stopnja
            n // 4,     # prehod
            n // 2,     # visoka koncentracija
            n - 1       # zvezda
        ]

    # vedno sortiramo Δ
    Delta_values = sorted(set(Delta_values))

    for alpha in alpha_values:
        alpha_R = RR(alpha)

        print("=" * 60)
        print(f"α = {float(alpha_R):.2f}")
        print("=" * 60)

        prev_tree = None   # seed reset za vsak α

        for Delta in Delta_values:
            print(f"Δ = {Delta}")

            # adaptivni parametri
            if n_starts is None or n_steps is None:
                n_s, n_p = adaptive_params(n, Delta)
            else:
                n_s, n_p = n_starts, n_steps

            best_val, best_tree = stochastic_search_fixed_n(
                n=n,
                Delta=Delta,
                alpha=alpha_R,
                n_starts=n_s,
                n_steps=n_p,
                start_tree=prev_tree
            )

            assert best_tree.is_tree()
            assert max_degree(best_tree) == Delta

            deg_seq = sorted(best_tree.degree())

            # >>> DODANO <<<
            count_max = deg_seq.count(Delta)

            # opcijsko: druga največja stopnja
            unique_degs = sorted(set(deg_seq))
            second_max = unique_degs[-2] if len(unique_degs) >= 2 else None

            print(f"  best value           = {best_val}")
            print(f"  degree seq           = {deg_seq}")
            print(f"  #vertices with deg Δ = {count_max}")
            if second_max is not None:
                print(f"  second max degree    = {second_max}")

            if print_edges:
                print(f"  edges                = {best_tree.edges(labels=False)}")

            print()

            prev_tree = best_tree   # seed za naslednji Δ

        print()


In [40]:
#n = 60
"""
alpha_values = [0.05, 0.45, 0.5, 0.55, 0.95]
Delta_values = [
    2,          # pot
    3, 4, 5,    # prvo razvejanje
    7, 10,      # srednje stopnje
    n//4,       # prehod
    n//2,       # visoka koncentracija
    n-1         # zvezda
]
"""

scan_alpha_and_delta(n=60)


α = 0.05
Δ = 2


  best value           = 65.4130566813360
  degree seq           = [1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
  #vertices with deg Δ = 58
  second max degree    = 1
  edges                = [(0, 1), (0, 2), (1, 4), (2, 3), (3, 6), (4, 5), (5, 7), (6, 8), (7, 11), (8, 9), (9, 10), (10, 13), (11, 12), (12, 14), (13, 18), (14, 15), (15, 16), (16, 17), (17, 23), (18, 19), (19, 20), (20, 21), (21, 22), (22, 24), (23, 26), (24, 25), (25, 31), (26, 27), (27, 28), (28, 29), (29, 30), (30, 32), (31, 34), (32, 33), (33, 36), (34, 35), (35, 37), (36, 39), (37, 38), (38, 43), (39, 40), (40, 41), (41, 42), (42, 45), (43, 44), (44, 48), (45, 46), (46, 47), (47, 50), (48, 49), (49, 51), (50, 53), (51, 52), (52, 55), (53, 54), (54, 59), (55, 56), (56, 57), (57, 58)]

Δ = 3
  best value           = 67.0538393584055
  degree seq           = [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
#scan_alpha_and_delta(n=123)

In [42]:
#random_tree_with_exact_max_degree(15, 6)
#zakomentiran da ga ne nariše

In [43]:
#random_bounded_degree_tree(15, 6)
#zakomentiran da ga ne nariše

In [44]:
# BREZ DELTA
"""
def random_tree_move2(G, max_attempts=50):
   
    Local move on a tree by relocating a leaf:
      - pick a random leaf ℓ with neighbor p,
      - detach ℓ from p,
      - attach ℓ to a different vertex q (≠ ℓ, ≠ p).

    This always preserves the tree property.
    No degree constraint is enforced.
    
    H = G.copy()

    for _ in range(max_attempts):
        # find all leaves
        leaves = [v for v in H.vertices() if H.degree(v) == 1]
        if not leaves:
            return G  # safety fallback

        leaf = random.choice(leaves)
        parent = H.neighbors(leaf)[0]

        # candidates: any vertex except leaf and its current parent
        candidates = [
            v for v in H.vertices()
            if v != leaf and v != parent
        ]

        if not candidates:
            continue

        new_parent = random.choice(candidates)

        # perform the move
        H.delete_edge(leaf, parent)
        H.add_edge(leaf, new_parent)

        # sanity check: should still be a tree
        if H.is_tree():
            return H

        # rollback if something went wrong (very unlikely)
        H.delete_edge(leaf, new_parent)
        H.add_edge(leaf, parent)

    # if all attempts fail, return original graph
    return G

"""

'\ndef random_tree_move2(G, max_attempts=50):\n   \n    Local move on a tree by relocating a leaf:\n      - pick a random leaf ℓ with neighbor p,\n      - detach ℓ from p,\n      - attach ℓ to a different vertex q (≠ ℓ, ≠ p).\n\n    This always preserves the tree property.\n    No degree constraint is enforced.\n    \n    H = G.copy()\n\n    for _ in range(max_attempts):\n        # find all leaves\n        leaves = [v for v in H.vertices() if H.degree(v) == 1]\n        if not leaves:\n            return G  # safety fallback\n\n        leaf = random.choice(leaves)\n        parent = H.neighbors(leaf)[0]\n\n        # candidates: any vertex except leaf and its current parent\n        candidates = [\n            v for v in H.vertices()\n            if v != leaf and v != parent\n        ]\n\n        if not candidates:\n            continue\n\n        new_parent = random.choice(candidates)\n\n        # perform the move\n        H.delete_edge(leaf, parent)\n        H.add_edge(leaf, new_parent)

In [45]:
"""

# BREZ DELTA
def stochastic_best_tree2(n, alpha, n_starts=20, n_steps=2000):
    
    Stochastic hill-climbing search for trees maximizing
    the general Sombor index among ALL n-vertex trees.

    Uses:
      - graphs.RandomTree(n) for random restarts (uniform random trees)
      - leaf-relocation local moves (Δ-free)

    Parameters
    ----------
    n : int
    alpha : float
    n_starts : int
        Number of random restarts.
    n_steps : int
        Number of local moves per restart.

    Returns
    -------
    (best_val, best_tree)
        best_val : RR
        best_tree : Graph
    
    best_val = -infinity
    best_tree = None

    alpha = RR(alpha)  # enforce numeric alpha

    for s in range(n_starts):

        # uniform random labelled tree (Sage built-in)
        G = graphs.RandomTree(n)
        val = general_sombor_index(G, alpha)

        local_best_val = val
        local_best_tree = G

        for step in range(n_steps):
            H = random_tree_move2(G)   # Δ-free local move
            if H is G:
                continue

            new_val = general_sombor_index(H, alpha)

            # hill-climbing acceptance rule
            if new_val >= val - 1e-12:
                G, val = H, new_val

                if val > local_best_val + 1e-12:
                    local_best_val = val
                    local_best_tree = G

        # update global best
        if local_best_val > best_val + 1e-12:
            best_val = local_best_val
            best_tree = local_best_tree

    return best_val, best_tree
"""

'\n\n# BREZ DELTA\ndef stochastic_best_tree2(n, alpha, n_starts=20, n_steps=2000):\n    \n    Stochastic hill-climbing search for trees maximizing\n    the general Sombor index among ALL n-vertex trees.\n\n    Uses:\n      - graphs.RandomTree(n) for random restarts (uniform random trees)\n      - leaf-relocation local moves (Δ-free)\n\n    Parameters\n    ----------\n    n : int\n    alpha : float\n    n_starts : int\n        Number of random restarts.\n    n_steps : int\n        Number of local moves per restart.\n\n    Returns\n    -------\n    (best_val, best_tree)\n        best_val : RR\n        best_tree : Graph\n    \n    best_val = -infinity\n    best_tree = None\n\n    alpha = RR(alpha)  # enforce numeric alpha\n\n    for s in range(n_starts):\n\n        # uniform random labelled tree (Sage built-in)\n        G = graphs.RandomTree(n)\n        val = general_sombor_index(G, alpha)\n\n        local_best_val = val\n        local_best_tree = G\n\n        for step in range(n_steps):\

In [46]:
# testiranje na primeru
#POPRAV ZA NOVO FUNKCIJO!!

n = 12
alpha = 0.5


# sistematično iskanje
exact_val, exact_list = systematic_best_trees_all(n, alpha)

# take the first optimal tree
exact_edges, exact_delta = exact_list[0]

# reconstruct graph to inspect degrees
G_exact = Graph()
G_exact.add_vertices(range(n))
G_exact.add_edges(exact_edges)

exact_degrees = sorted(G_exact.degree())

print("EXACT search:")
print("  best value    =", exact_val)
print("  max degree Δ  =", exact_delta)
print("  degree seq    =", exact_degrees)


# stohastično iskanje
stoch_val, stoch_tree = stochastic_best_tree2(
    n=n,
    alpha=alpha,
    n_starts=10,
    n_steps=2000
)

stoch_degrees = sorted(stoch_tree.degree())
stoch_delta = max(stoch_tree.degree())

print("\nSTOCHASTIC search:")
print("  best value    =", stoch_val)
print("  max degree Δ  =", stoch_delta)
print("  degree seq    =", stoch_degrees)




EXACT search:
  best value    = 121.498971189060
  max degree Δ  = 11
  degree seq    = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11]


NameError: name 'stochastic_best_tree2' is not defined

In [ ]:
# samo stohastično iskanje za velik n
# ne pride zvezda!! (zakaj?)
#pride vsakič mal drugač ampak to je pomoje ok?

"""
n = 55
alpha = 0.5

stoch_val, stoch_tree = stochastic_best_tree2(
    n=n,
    alpha=alpha,
    n_starts=10,
    n_steps=2000
)

stoch_degrees = sorted(stoch_tree.degree())
stoch_delta = max(stoch_tree.degree())

print("\nSTOCHASTIC search:")
print("  best value    =", stoch_val)
print("  max degree Δ  =", stoch_delta)
print("  degree seq    =", stoch_degrees)

"""

'\nn = 55\nalpha = 0.5\n\nstoch_val, stoch_tree = stochastic_best_tree2(\n    n=n,\n    alpha=alpha,\n    n_starts=10,\n    n_steps=2000\n)\n\nstoch_degrees = sorted(stoch_tree.degree())\nstoch_delta = max(stoch_tree.degree())\n\nprint("\nSTOCHASTIC search:")\nprint("  best value    =", stoch_val)\nprint("  max degree Δ  =", stoch_delta)\nprint("  degree seq    =", stoch_degrees)\n\n'

In [ ]:
# nevem kaj je to, pomoje se lahko zbriše
"""res = classify_trees_by_max_degree(8)

for delta, treelist in sorted(res.items()):
    print(f"Δ = {delta}")
    for G in treelist:
        print("  edges:", G.edges(labels=False))

"""

'res = classify_trees_by_max_degree(8)\n\nfor delta, treelist in sorted(res.items()):\n    print(f"Δ = {delta}")\n    for G in treelist:\n        print("  edges:", G.edges(labels=False))\n\n'

In [ ]:
#systematic_best_trees_all(4, alpha=0.2)

In [ ]:
"""
best_val, best_list = systematic_best_trees_all(10, alpha=0.2)

print("Največji Sombor indeks =", best_val)
for edges, delta in best_list:
    print("edges:", edges)
    print("delta =", delta)

"""

'\nbest_val, best_list = systematic_best_trees_all(10, alpha=0.2)\n\nprint("Največji Sombor indeks =", best_val)\nfor edges, delta in best_list:\n    print("edges:", edges)\n    print("delta =", delta)\n\n'

In [ ]:
"""
res = best_sombor_by_delta(8, alpha=1)

for delta in sorted(res.keys()):
    print(f"Δ = {delta}")
    print("  največji Sombor indeks =", res[delta]["best_val"])
    for edges, d in res[delta]["best_trees"]:
        print("  edges:", edges)
    print()

"""

'\nres = best_sombor_by_delta(8, alpha=1)\n\nfor delta in sorted(res.keys()):\n    print(f"Δ = {delta}")\n    print("  največji Sombor indeks =", res[delta]["best_val"])\n    for edges, d in res[delta]["best_trees"]:\n        print("  edges:", edges)\n    print()\n\n'